In [1]:
import requests
import pandas as pd
from time import sleep
import datetime
from tqdm.notebook import tqdm

In [20]:
ibdata = pd.read_csv('isin_dropdup_17697_final_Jiarui.csv',index_col=0)
ibdata=ibdata.dropna(subset=['secIdList'])
ibdata

,longName,marketName,secIdList,Origin_Name,Origin_Stock,Origin_Country
new_index,,,,,,
0,1ST GROUP LTD,1ST,AU0000001ST0,1ST GROUP LTD,1ST,Australia
1,2U INC,NMS,US90214J1016,2U INC,TWOU,United States of America
2,3DX INDUSTRIES INC,CURRENT,US88556A1007,3DX INDUSTRIES INC,DDDX,United States of America
3,3M CO,MMM,US88579Y1010,3M CO,MMM,Mexico
4,88 ENERGY LTD,88E,AU00000088E2,88 ENERGY LTD,88E,United Kingdom
...,...,...,...,...,...,...
268,WISDOMTREE PHYSICAL GOLD,ETF,DE000A0N62G0,WISDOMTREE METAL SECURITIES,VZLD,NaN
269,WYNN RESORTS LTD.,NMS,US9831341071,WYNN RESORTS LTD,WYNN,NaN
270,XERIS BIOPHARMA HOLDINGS INC,NMS,US98422E1038,XERIS PHARMACEUTICALS INC,XERS,United States of America


In [3]:
openfigi_apikey = ''#own apikey to increase limit
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [4]:
def map_jobs(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()

In [5]:
figi_data=pd.DataFrame(columns=['figi', 'name', 'ticker', 'exchCode', 'compositeFIGI', 'securityType',
       'marketSector', 'shareClassFIGI', 'securityType2',
       'securityDescription'])

In [6]:
empty= {'figi': '',
  'name': '',
  'ticker': '',
  'exchCode': '',
  'compositeFIGI': '',
  'securityType': '',
  'marketSector': '',
  'shareClassFIGI': '',
  'securityType2': '',
  'securityDescription': ''}

In [7]:
for i in tqdm(range(707)):
    isins = ibdata['secIdList'][25*i:25*i+25]
    jobs=[]
    for n in isins:
        newjob = {'idType': 'ID_ISIN', 'idValue': n},
        jobs.extend(newjob)
    job_results = map_jobs(jobs)
    just_dictionaries=[]
    for j in job_results:
        try:
            just_dictionaries.append(j['data'][0])
        except:
            just_dictionaries.append(empty)
    figi = pd.DataFrame.from_dict(just_dictionaries)
    figi_data=pd.concat([figi_data,figi])
    sleep(6)

  0%|          | 0/707 [00:00<?, ?it/s]

In [8]:
isins = ibdata['secIdList'][17675:]
jobs=[]
for n in isins:
    newjob = {'idType': 'ID_ISIN', 'idValue': n},
    jobs.extend(newjob)
job_results = map_jobs(jobs)
just_dictionaries=[]
for j in job_results:
    try:
        just_dictionaries.append(j['data'][0])
    except:
        just_dictionaries.append(empty)
figi = pd.DataFrame.from_dict(just_dictionaries)
figi_data=pd.concat([figi_data,figi])
sleep(6)

In [9]:
figi_data=figi_data.add_prefix('Open_FIGI')
figi_data

,figi,name,ticker,exchCode,compositeFIGI,securityType,marketSector,shareClassFIGI,securityType2,securityDescription,uniqueID,uniqueIDFutOpt
0,BBG008HNLRF4,1ST GROUP LTD,1ST,AU,BBG008HNLRF4,Common Stock,Equity,BBG008HNLRG3,Common Stock,1ST,NaN,NaN
1,BBG001KS9450,2U INC,TWOU,US,BBG001KS9450,Common Stock,Equity,BBG001V0RR30,Common Stock,TWOU,NaN,NaN
2,BBG000C129N6,3DX INDUSTRIES INC,DDDX,US,BBG000C129N6,Common Stock,Equity,BBG001STRH79,Common Stock,DDDX,NaN,NaN
3,BBG000BP52R2,3M CO,MMM,US,BBG000BP52R2,Common Stock,Equity,BBG001S5T7X2,Common Stock,MMM,NaN,NaN
4,BBG000BWTTK9,88 ENERGY LTD,POQ,GR,BBG000BWTTK9,Common Stock,Equity,BBG001SCV1F7,Common Stock,POQ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16,BBG000RCM151,WISDOMTREE PHYSICAL GOLD,VZLD,GR,BBG000RCM151,ETP,Equity,BBG001STG185,Mutual Fund,VZLD,None,None
17,BBG000LD9JQ8,WYNN RESORTS LTD,WYNN,US,BBG000LD9JQ8,Common Stock,Equity,BBG001SJZ2V8,Common Stock,WYNN,None,None
18,BBG012V1MG52,XERIS BIOPHARMA HOLDINGS INC,XERS,US,BBG012V1MG52,Common Stock,Equity,BBG012V1MG61,Common Stock,XERS,None,None
19,BBG000C6KC85,NOVACCESS GLOBAL INC,XSNX,US,BBG000C6KC85,Common Stock,Equity,BBG001SC0BS5,Common Stock,XSNX,None,None


In [10]:
figi_data['new_index']=[i for i in range(len(figi_data))]
figi_data=figi_data.set_index(['new_index'])
ibdata['new_index']=[i for i in range(len(ibdata))]
ibdata=ibdata.set_index(['new_index'])

In [12]:
isin_figi=pd.concat([figi_data,ibdata],axis=1)
isin_figi

,figi,name,ticker,exchCode,compositeFIGI,securityType,marketSector,shareClassFIGI,securityType2,securityDescription,uniqueID,uniqueIDFutOpt,longName,marketName,secIdList,Origin_Name,Origin_Stock,Origin_Country
new_index,,,,,,,,,,,,,,,,,,
0,BBG008HNLRF4,1ST GROUP LTD,1ST,AU,BBG008HNLRF4,Common Stock,Equity,BBG008HNLRG3,Common Stock,1ST,NaN,NaN,1ST GROUP LTD,1ST,AU0000001ST0,1ST GROUP LTD,1ST,Australia
1,BBG001KS9450,2U INC,TWOU,US,BBG001KS9450,Common Stock,Equity,BBG001V0RR30,Common Stock,TWOU,NaN,NaN,2U INC,NMS,US90214J1016,2U INC,TWOU,United States of America
2,BBG000C129N6,3DX INDUSTRIES INC,DDDX,US,BBG000C129N6,Common Stock,Equity,BBG001STRH79,Common Stock,DDDX,NaN,NaN,3DX INDUSTRIES INC,CURRENT,US88556A1007,3DX INDUSTRIES INC,DDDX,United States of America
3,BBG000BP52R2,3M CO,MMM,US,BBG000BP52R2,Common Stock,Equity,BBG001S5T7X2,Common Stock,MMM,NaN,NaN,3M CO,MMM,US88579Y1010,3M CO,MMM,Mexico
4,BBG000BWTTK9,88 ENERGY LTD,POQ,GR,BBG000BWTTK9,Common Stock,Equity,BBG001SCV1F7,Common Stock,POQ,NaN,NaN,88 ENERGY LTD,88E,AU00000088E2,88 ENERGY LTD,88E,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17691,BBG000RCM151,WISDOMTREE PHYSICAL GOLD,VZLD,GR,BBG000RCM151,ETP,Equity,BBG001STG185,Mutual Fund,VZLD,None,None,WISDOMTREE PHYSICAL GOLD,ETF,DE000A0N62G0,WISDOMTREE METAL SECURITIES,VZLD,NaN
17692,BBG000LD9JQ8,WYNN RESORTS LTD,WYNN,US,BBG000LD9JQ8,Common Stock,Equity,BBG001SJZ2V8,Common Stock,WYNN,None,None,WYNN RESORTS LTD.,NMS,US9831341071,WYNN RESORTS LTD,WYNN,NaN
17693,BBG012V1MG52,XERIS BIOPHARMA HOLDINGS INC,XERS,US,BBG012V1MG52,Common Stock,Equity,BBG012V1MG61,Common Stock,XERS,None,None,XERIS BIOPHARMA HOLDINGS INC,NMS,US98422E1038,XERIS PHARMACEUTICALS INC,XERS,United States of America


In [16]:
n=0
for i in range(len(isin_figi)):
    if figi_data.figi[i]=='':
        n+=1
n

1938

In [18]:
isin_figi.to_csv('isin_figi_17696_empty1938.csv')